# Import the necessary libraries

In [1]:
import gc
import os
import cv2 
import random
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from sklearn.preprocessing import StandardScaler


import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet152 # for feature extraction
from torch.utils.data import DataLoader # for batching data
from torch.utils.data import TensorDataset 
from torchvision.transforms import transforms 

warnings.filterwarnings('ignore')

# Load the Feature Extractor

In [2]:
# Deep_features = []
# root = 'C:\\Users\\Eurus\\Desktop\\Data\\images'
device = torch.device('cuda')

# # Load the ResNet-152 model
# resnet = resnet152(pretrained=False)

In [3]:
# resnet.to(device)
# resnet.eval()

# Prepare the Data

In [4]:
# # Transformation for preprocessing
# preprocess = transforms.Compose([
#     transforms.Resize((540, 540)),
#     transforms.ToTensor()
# ])

In [5]:
# def feature_extractor(root):
#     # Iterate over the images
#     for idx in tqdm(os.listdir(root)):
        
#         # Read the image
#         path = os.path.join(root, idx)
#         img = Image.open(path)
        
#         # Preprocess the images (Normalize-> add a dummy dimention -> move it to cuda)
#         img_tensor = preprocess(img).div(255.0).unsqueeze(0).to(device)
        
#         # Extract features using ResNet-152
#         with torch.no_grad():
#             features = resnet(img_tensor)
            
#         # Flatten the features
#         features = features.flatten().cpu().numpy()
        
#         # Store the features in the dictionary
#         if "T" in idx:
#             Deep_features.append({'Label':'T','Features':features})
#         elif "N" in idx:
#             Deep_features.append({'Label':'N','Features':features})
#         else:
#             print("error!")
            
    
#     return Deep_features
        

In [6]:
# Train_features = feature_extractor(root)

In [7]:
# shuffle the data
# random.shuffle(Train_features)

In [8]:
# "Cleaning feature titles to make the dataset easy to use"

# df = pd.DataFrame()

# # Iterate over the list of dictionaries
# for idx, d in enumerate(Train_features):
#     # Extract the label and features from each dictionary
#     label = d['Label']
#     features = d['Features']
    
#     # Create a dictionary for the row data
#     row_data = {'Label': label}
    
#     # Add the features as columns to the row dictionary
#     for i, value in enumerate(features):
#         column_name = f'F_{i}'
#         row_data[column_name] = value
    
#     # Append the row to the DataFrame
#     df = df.append(row_data, ignore_index=True)

    
# # Now we have df, Delete the residual
# del Train_features

# # Observe the results
# df.head(5)

In [9]:
# "Encoding the Labels"

# for row in tqdm(range(len(df))):
#     if df.loc[row,'Label'] == "N":
#         df.loc[row,'Label'] = 0
        
#     elif df.loc[row,'Label'] == "T":
#         df.loc[row,'Label'] = 1

# # Convert them all to float32        
# df = df.astype('float32')
# df.info()        

In [10]:
# # Beware !!!
# df.to_csv("df_output_cell_10.csv",index=False)

In [11]:
df = pd.read_csv("df_output_cell_10.csv")
df.head()

,Label,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,...,F_990,F_991,F_992,F_993,F_994,F_995,F_996,F_997,F_998,F_999
0,0.0,-3155.3340,2247.9016,-15242.063,2105.8013,4712.2056,26482.980,29321.460,22290.460,12380.1455,...,24386.512,-22758.016,32603.027,-22886.885,22423.560,-25667.980,7625.3670,9067.743,13116.528,300.804350
1,0.0,-4456.2400,5013.8184,-19840.195,3260.8843,5470.0503,33030.970,37948.010,27413.734,15669.2110,...,31678.412,-29751.850,42726.203,-28719.370,28016.207,-33457.973,11405.8260,12074.846,16793.018,76.287735
2,1.0,-3624.0283,4096.3154,-17587.424,2470.3438,5148.6680,28676.377,33115.723,23879.389,13839.6810,...,27025.328,-26016.256,36537.504,-24437.940,24452.035,-28386.310,9967.6120,10893.868,14938.218,215.583150
3,0.0,-4195.8760,4789.3535,-18938.963,2818.3047,5316.8325,31045.184,36170.170,26332.742,15174.0280,...,29719.297,-28442.473,40481.420,-27291.717,26547.879,-31284.033,10983.9375,11585.030,15812.540,209.395650
4,0.0,-5584.1300,6379.3710,-23731.717,3972.9082,6766.4370,39294.560,46416.805,32344.842,18326.2460,...,38469.840,-36466.477,51717.152,-34935.880,33479.850,-41357.324,14098.4250,13870.688,19981.975,21.897598


# Beware ! Error Zone

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
Cell In[21], line 18
     15 optimizer.zero_grad()
     17 # Forward Pass
---> 18 outputs = model(inputs)
     19 loss = criterion(outputs.squeeze(), labels)
     21 # Backpropagate

File ~\Desktop\YoloV8\Pytorch\lib\site-packages\torch\nn\modules\module.py:1501, in Module._call_impl(self, *args, **kwargs)
   1496 # If we don't have any hooks, we want to skip the rest of the logic in
   1497 # this function, and just call forward.
   1498 if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks or self._forward_pre_hooks
   1499         or _global_backward_pre_hooks or _global_backward_hooks
   1500         or _global_forward_hooks or _global_forward_pre_hooks):
-> 1501     return forward_call(*args, **kwargs)
   1502 # Do not call functions when jit is used
   1503 full_backward_hooks, non_full_backward_hooks = [], []

Cell In[16], line 98, in TabularTransformer.forward(self, x, memory)
     95 embedded = self.embedding(x)
     97 print(f" This is embeded before long:\n {embedded}")
---> 98 embedded = self.embedding.weight[embedded.long(), :].to_dense()
     99 print(f" This is embeded after long:\n {embedded}")
    100 embedded = embedded.permute(2, 0, 1)       

IndexError: index -18739 is out of bounds for dimension 0 with size 128

#### We must use scalers to rescale the data so it fits the dimention required by the Transformer

In [12]:
# cols = df.columns[1:]

# plt.figure()
# fig, ax = plt.subplots(15, 2 ,figsize=(21, 36))

# j = 1
# for col in cols[0:30]:
#     plt.subplot(15, 2, j)
#     sns.histplot(df[col], kde=True, bins=10000, label=col, color='Red')
#     plt.xlabel(col, fontsize=9); 
#     plt.rcParams['axes.facecolor'] = 'black'
#     plt.legend()
#     j += 1

# plt.show();

In [13]:
# df.describe()

In [14]:
# min_list = []
# max_list = []

# for col in tqdm(cols):
#     min_list.append(df[col].min())
#     max_list.append(df[col].max())


In [15]:
# plt.figure()
# fig, ax = plt.subplots(2, 1 ,figsize=(16, 8))

# for i in range(1, 3):
#     if i == 1:
#         plt.subplot(2, 1, i)
#         sns.histplot(min_list, kde=True,bins=1000, label="Features Min/Max Values Distribution", color='red')
#         plt.rcParams['axes.facecolor'] = 'black'
#         plt.xlabel("Min Values", fontsize=12, color='red')
# #         plt.legend()
        
#     elif i == 2:
#         plt.subplot(2, 1, i)
#         sns.histplot(max_list, kde=True,bins=1000, label="Features Min/Max Values Distribution", color='Green')
#         plt.rcParams['axes.facecolor'] = 'black'
#         plt.xlabel("Max Values", fontsize=12, color='Green')
# #         plt.legend()
    
#     else:
#         break;
        
# plt.show()

The First approach is going to be normalizing tha values of each column to be between -1,1

In [16]:
def ScaleMe(input_value, init_min, init_max, goal_min, goal_max):
    sclaled_value = ( ( (input_value - init_min) / (init_max - init_min) ) * (goal_max - goal_min) ) + goal_min
    
    return sclaled_value

#### A simple test ...
- Temp_vector = [1,2,3,4,5]
- Goal_vecor = [10,20,30,40,50]

In [17]:
# Temp_vector = [1,2,3,4,5] # --- > [10,20,30,40,50]
# Goal_vecor = []

# init_min = min(Temp_vector)
# init_max = max(Temp_vector)
# goal_min = 10
# goal_max = 50

# for idx in Temp_vector:
#     idx = ScaleMe(idx, init_min, init_max, goal_min, goal_max)
# #     Goal_vecor.append(int(Temp_var)) 

# print(Temp_vector)
    

OK, now we are going to apply it on each column of data frame seprately

In [18]:
init_min = df.iloc[:,1:].min()
init_max = df.iloc[:,1:].max()

goal_min = -1
goal_max = 1

# Apply the ScaleMe function to each column
scaled_df = pd.DataFrame()
for column in tqdm(df.iloc[:,1:].columns):
    df[column] = ScaleMe(df[column], init_min[column], init_max[column], goal_min, goal_max)

df

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 82.75it/s]


,Label,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,...,F_990,F_991,F_992,F_993,F_994,F_995,F_996,F_997,F_998,F_999
0,0.0,0.277372,-0.615168,0.224336,-0.445735,-0.269516,-0.171223,-0.222884,-0.178833,-0.181066,...,-0.204326,0.254114,-0.233859,0.179850,-0.196260,0.210875,-0.377951,-0.351414,-0.256950,0.378346
1,0.0,-0.130541,0.041848,-0.134103,-0.160929,-0.059789,0.131267,0.119419,0.103986,0.134152,...,0.142357,-0.099134,0.119599,-0.128862,0.110622,-0.137260,0.095425,0.045196,0.096342,0.105787
2,1.0,0.130408,-0.176096,0.041508,-0.355850,-0.148729,-0.069897,-0.072327,-0.091120,-0.041187,...,-0.078868,0.089545,-0.096484,0.097753,-0.084953,0.089393,-0.084663,-0.110564,-0.081895,0.274889
3,0.0,-0.048901,-0.011471,-0.063849,-0.270054,-0.102191,0.039532,0.048874,0.044312,0.086695,...,0.049213,-0.032999,0.041221,-0.053297,0.030051,-0.040106,0.042598,-0.019406,0.002123,0.267378
4,0.0,-0.484203,0.366222,-0.437459,0.014633,0.298974,0.420619,0.455462,0.376196,0.388797,...,0.465246,-0.438279,0.433525,-0.457899,0.410426,-0.490281,0.432583,0.282052,0.402785,0.039758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13827,0.0,-0.028082,-0.106008,-0.106702,-0.160814,0.058581,0.091861,0.095292,0.073248,0.085436,...,0.104352,-0.068842,0.077622,-0.073891,0.075323,-0.078415,0.060094,0.015908,0.073718,-0.350603
13828,0.0,-0.043126,0.183206,0.004159,-0.002382,-0.175601,0.004300,0.033599,-0.003198,-0.016766,...,0.043652,-0.002709,0.016395,-0.010907,0.002114,-0.048727,0.036789,-0.125225,-0.050151,0.306791
13829,0.0,-0.440252,0.093702,-0.261933,-0.404802,0.160788,0.199729,0.250910,0.195131,0.255778,...,0.244014,-0.235995,0.204007,-0.245006,0.203682,-0.245359,0.312797,0.083168,0.192885,0.176919
13830,0.0,0.483210,-0.208352,0.146442,-0.136143,-0.177333,-0.166753,-0.156148,-0.197578,-0.171186,...,-0.183973,0.175960,-0.161631,0.192254,-0.166952,0.181626,-0.148129,-0.098948,-0.181747,-0.214113


In [19]:
df['Label'] = df['Label'].astype('int32')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13832 entries, 0 to 13831
Columns: 1001 entries, Label to F_999
dtypes: float64(1000), int32(1)
memory usage: 105.6 MB


In [21]:
df.describe()

,Label,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,...,F_990,F_991,F_992,F_993,F_994,F_995,F_996,F_997,F_998,F_999
count,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,...,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000,13832.000000
mean,0.091888,0.050002,-0.146251,-0.023052,-0.234028,-0.058076,0.011126,0.014331,-0.010526,0.012198,...,0.015931,0.004815,-0.000148,-0.009463,0.002045,-0.015677,-0.011731,-0.042348,-0.006613,0.030028
std,0.288879,0.380545,0.396829,0.329379,0.288576,0.303694,0.312743,0.332670,0.304844,0.307483,...,0.330350,0.328886,0.325247,0.323179,0.319912,0.330032,0.369707,0.318389,0.328246,0.245453
min,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,-0.222134,-0.497614,-0.262873,-0.462236,-0.259246,-0.209042,-0.234745,-0.222203,-0.200742,...,-0.227611,-0.237108,-0.241835,-0.247751,-0.225891,-0.258149,-0.312477,-0.262782,-0.243810,-0.124988
50%,0.000000,0.020511,-0.061110,-0.051866,-0.257789,-0.036339,0.027929,0.044811,0.009569,0.040671,...,0.043060,-0.023309,0.026419,-0.027280,0.021489,-0.034050,0.047533,-0.019619,0.014742,0.028527
75%,0.000000,0.374754,0.148974,0.213304,-0.050767,0.145703,0.237206,0.257521,0.210005,0.236339,...,0.258834,0.250943,0.237142,0.222506,0.233489,0.229759,0.263455,0.169665,0.228592,0.200233
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
df.tail()

,Label,F_0,F_1,F_2,F_3,F_4,F_5,F_6,F_7,F_8,...,F_990,F_991,F_992,F_993,F_994,F_995,F_996,F_997,F_998,F_999
13827,0,-0.028082,-0.106008,-0.106702,-0.160814,0.058581,0.091861,0.095292,0.073248,0.085436,...,0.104352,-0.068842,0.077622,-0.073891,0.075323,-0.078415,0.060094,0.015908,0.073718,-0.350603
13828,0,-0.043126,0.183206,0.004159,-0.002382,-0.175601,0.004300,0.033599,-0.003198,-0.016766,...,0.043652,-0.002709,0.016395,-0.010907,0.002114,-0.048727,0.036789,-0.125225,-0.050151,0.306791
13829,0,-0.440252,0.093702,-0.261933,-0.404802,0.160788,0.199729,0.250910,0.195131,0.255778,...,0.244014,-0.235995,0.204007,-0.245006,0.203682,-0.245359,0.312797,0.083168,0.192885,0.176919
13830,0,0.483210,-0.208352,0.146442,-0.136143,-0.177333,-0.166753,-0.156148,-0.197578,-0.171186,...,-0.183973,0.175960,-0.161631,0.192254,-0.166952,0.181626,-0.148129,-0.098948,-0.181747,-0.214113
13831,0,0.003372,-0.038983,-0.030510,-0.247598,-0.111729,0.005667,0.016766,0.002611,0.035085,...,0.018008,-0.002359,0.013903,-0.011962,-0.007253,-0.012366,0.004602,-0.038559,-0.016745,0.177205


Now lets see what its looks like

In [23]:
# del min_list, max_list

In [24]:
# min_list = []
# max_list = []
# cols = df.columns[1:]

# for col in tqdm(cols):
#     min_list.append(df[col].min())
#     max_list.append(df[col].max())

In [25]:
# cols = df.columns[1:]

# plt.figure()
# fig, ax = plt.subplots(15, 2 ,figsize=(21, 36))

# j = 1
# for col in cols[0:30]:
#     plt.subplot(15, 2, j)
#     sns.histplot(df[col], kde=True, bins=1000, label=col, color='Red')
#     plt.xlabel(col, fontsize=9); 
#     plt.rcParams['axes.facecolor'] = 'black'
#     plt.legend()
#     j += 1

# plt.show();

# Train Test Split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df['Label'], test_size=0.2, random_state=42, stratify=df['Label'])

# Delete the residual
del df 
gc.collect()

0

In [27]:
# Load them as PyTorch Tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [28]:
# Create DataLoader objects for batching the data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [29]:
# Create DataLoader objects for batching the data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=True)

# Define the Tabular Transformer module

In [123]:
class TabularTransformer(nn.Module):
    def __init__(
        self,
        input_dim,                   # Total number of Feature columns
        hidden_dim,                  # Neuron Density of MLP Layers
        output_dim,                  # Total number of Label columns
        num_attention_heads,         # Total Number of attention heads of Transformer
        num_transformer_layers,      # Total Number of transformer layers
        num_mlp_layers,              # Total number of MLP layers
        middle_activation,           # MLP layer Activation Function
        last_activation,             # Last layer Activation Function
        dropout_prob,                # Dropout possibility of Dropout layers (float)
        pooling_type                 # Pooling Strategy : mean(Average) , max(Maximum), global(mean)
    ):
        super(TabularTransformer, self).__init__()
        
        # New Line
        self.pooling_type = pooling_type
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        
        
        """Should I remove this because of the situation with the data in the next steps"""
        # Embedding layer
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        
        # Transformer layers
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model = hidden_dim,
                nhead = num_attention_heads,
            ),
            num_layers = num_transformer_layers
        )
        
        # MLP Layers
        self.mlp = nn.Sequential()
        for i in range(num_mlp_layers):
            # Last FC Layer
            if i == num_mlp_layers-1:
                self.mlp.add_module(
                    f"FC_{i+1}",
                    nn.Linear(hidden_dim, output_dim)
                )
                   
                
                ######## Suspicion Area ##########
            else:
                self.mlp.add_module(
                    f"FC_{i+1}",
                    nn.Linear(hidden_dim,hidden_dim)
                )    
                ######## Suspicion Area ##########
                
                
            # Activation Functions for Final FC Layer
            if i == num_mlp_layers-1:
                if last_activation == "sigmoid":
                    self.mlp.add_module(
                        f"Activation_{i+1}",
                        nn.Sigmoid()
                    )
                elif last_activation == "softmax":
                    self.mlp.add_module(
                        f"Activation_{i+1}",
                        nn.Softmax(dim=-1)
                    )
                else:
                    raise ValueError(f"Invalid activation function for the last layer: {last_activation}")
            
            # Activation Functions for other FC layers
            else:
                if middle_activation == "relu":
                    self.mlp.add_module(
                        f"Activation_{i+1}",
                        nn.ReLU()
                    )
                elif middle_activation == "leakyrelu":
                    self.mlp.add_module(
                        f"Activation_{i+1}",
                        nn.LeakyReLU()
                    )
                else:
                    raise ValueError(f"Invalid activation function for the middle layers : {last_activation} ")
            
            # Dropout Layer
            if i != num_mlp_layers-1:
                if dropout_prob > 0.0:
                    self.mlp.add_module(
                        f"Dropout_{i+1}",
                        nn.Dropout(dropout_prob)
                    )
       
                    
    def forward(self,  x, memory=None): 
        
        print(x.shape)
        # Apply linear transformation
        embedded = self.embedding(x)    
        """This line seems meaningless to me"""
#         embedded = self.embedding.weight[embedded.long(), :].to_dense()
        print(embedded.shape)
        embedded = embedded.unsqueeze(0)
        print(embedded.shape)
        embedded = embedded.permute(1, 0, 2)   
#         memory = torch.rand(input_dim, 1, hidden_dim).to(device)
        memory = torch.rand(self.input_dim, x.size(0), self.hidden_dim).to(device)
        transformer_output = self.transformer(embedded, memory)
    
#         # Reset the dimensions to original order for upcoming FC/Pooling Layers
        transformer_output = transformer_output.permute(1, 0, 2)
        
        # Pooling Layer
        if self.pooling_type == "mean":
            pooled_output = torch.mean(transformer_output, dim=1)
        elif self.pooling_type == "max":
            pooled_output = torch.max(transformer_output, dim=1) 
        else:
            raise ValueError(f"Invalid pooling_type: {self.pooling_type}")
        
        pooled_output = pooled_output.permute(1,0)
        mlp_output = self.mlp(pooled_output) 
        

        return mlp_output
                    

# Model Instantiation

In [124]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cuda")

In [125]:
model = TabularTransformer(
    input_dim = 1000,
    hidden_dim = 128,
    output_dim = 1,
    num_attention_heads = 8,
    num_transformer_layers = 2,
    num_mlp_layers = 4,
    middle_activation = 'relu',
    last_activation = 'sigmoid',
    dropout_prob = 0.1,
    pooling_type = 'mean'

).to(device)


In [126]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [127]:
temp = torch.rand(8, 1000, dtype=torch.float32).to(device)
model(temp)
# mlp_output = model(temp)
# # Apply threshold (0.5) to convert probabilities to binary values
# binary_output = (mlp_output >= 0.5).float()
# binary_output

torch.Size([8, 1000])


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [ ]:
num_epochs = 30
model.train()

init_outputs_list = []
final_outputs_list = []

for epoch in tqdm(range(num_epochs)):
    print(f"Epoch {epoch} in progress ...\n")
#     running_loss = 0.0
    TP = 0
    FN = 0
    FP = 0
    
    for inputs, labels in train_loader:
        # All set to work on same device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Reset the Gradients
        optimizer.zero_grad()
        
        # Forward Pass
        outputs = model(inputs)
        
        loss = criterion(outputs.squeeze(), labels.squeeze())
        
        # Backpropagate
        loss.backward()
        
        # Update the weights and loss
        optimizer.step()
#         running_loss += loss.item()


    """Evaluation"""
    model.eval().to(device)
    
    with torch.no_grad():
        for test_inputs, test_labels in test_loader:
            test_inputs = test_inputs.to(device)
            test_labels = test_labels.to(device)
            
            test_outputs = model(test_inputs)
            binary_predictions = (test_outputs >= 0.5).float()
            
            test_labels = test_labels.squeeze()
            val_loss = criterion(test_outputs.squeeze(), test_labels)
            
            # Calculate the number of true positives, false negatives, and false positives
            TP = ((binary_predictions == 1.0) & (test_labels == 1.0)).sum().item()
            FN = ((binary_predictions == 0.0) & (test_labels == 1.0)).sum().item()
            FP = ((binary_predictions == 1.0) & (test_labels == 0.0)).sum().item()
            
            loss = criterion(binary_predictions.squeeze(), test_labels.squeeze())
        
        
        # Metric Calculations
        if TP == 0 and FN == 0:  # (Recall is 0/0)
            recall = 0.0
            precision = 0.0
            f1_score = 0.0

        elif TP == 0 and FP == 0:  # (Precision is 0/0)
            recall = 0.0
            precision = 0.0
            f1_score = 0.0

        else:
            recall = TP/(TP+FN)
            precision = TP/(TP+FP)
            f1_score = (2*precision*recall)/(precision+recall)
        
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {val_loss:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1_score:.4f}")
        

# Checking the output values Distribution

In [ ]:
# del init_outputs_list, final_outputs_list

In [ ]:
# len(final_outputs_list)

In [ ]:
# plt.figure()
# fig, ax = plt.subplots(1, 1 ,figsize=(16, 5))

# for i in range(1, 2):
#     if i == 1:
#         plt.subplot(1, 1, i)
#         sns.histplot(init_outputs_list, kde=True,bins=11065, label="Model Output Values", color='red')
#         plt.rcParams['axes.facecolor'] = 'black'
#         plt.xlabel("Model 1st epoch Output Values", fontsize=12, color='red')
    
#     else:
#         break;
        
# plt.show()

In [ ]:
plt.figure(figsize=(16, 5))

# Plotting the first list
plt.subplot(1, 1, 1)
sns.histplot(init_outputs_list, kde=True, bins=11065, label="Model Output Values", color='red')
plt.rcParams['axes.facecolor'] = 'black'
plt.xlabel("Model 1st epoch Output Values", fontsize=12, color='red')

# Plotting the second list
plt.hist(final_outputs_list, kde=True, bins=11065, label="Second List Values", color='blue')
plt.xlabel("Second List Values", fontsize=12, color='blue')

plt.legend()  # Show legend for both plots
plt.show()